In [40]:
import pandas as pd
import numpy as np
from datetime import datetime
from pytz import timezone
import pytz


In [6]:
Data = pd.read_csv('UberPriceTrackXTest.csv')
Data.head()

,Date,Unnamed: 1,Price,Time,Miles,Wait_Time
0,2018-11-08 17:51:51.493805,1541699632,8.71,960,3.27,4
1,2018-11-08 17:55:56.128564,1541699877,8.71,960,3.27,3
2,2018-11-08 17:56:57.918118,1541699939,8.71,960,3.27,2
3,2018-11-08 17:57:59.659881,1541700001,8.71,960,3.27,2
4,2018-11-08 17:59:01.625413,1541700063,8.71,960,3.27,4


In [31]:
def time_convert(dt):
    date_format='%Y-%m-%d %H:%M:%S.%f'

    date_current_utc = datetime.strptime(date, date_format)
    date_current_utc = pytz.utc.localize(date_current_utc)

    date_changed = date_current_utc.astimezone(timezone('US/Pacific'))
    date_changed = date_changed.strftime(date_format)

    return date_changed


In [39]:
date = list(Data['Date'])

converted_date = []
for date in date:
    converted_date.append(time_convert(date))
    
Data['Converted Date'] = converted_date
Data_df = Data[['Date','Converted Date','Price','Time','Miles','Wait_Time']]
Data_df.head()

,Date,Converted Date,Price,Time,Miles,Wait_Time
0,2018-11-08 17:51:51.493805,2018-11-08 09:51:51.493805,8.71,960,3.27,4
1,2018-11-08 17:55:56.128564,2018-11-08 09:55:56.128564,8.71,960,3.27,3
2,2018-11-08 17:56:57.918118,2018-11-08 09:56:57.918118,8.71,960,3.27,2
3,2018-11-08 17:57:59.659881,2018-11-08 09:57:59.659881,8.71,960,3.27,2
4,2018-11-08 17:59:01.625413,2018-11-08 09:59:01.625413,8.71,960,3.27,4
